In [18]:
import sys
import nltk
import numpy
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np
import multiprocessing

from gensim.models import Word2Vec

def token_with_punc(text):
    # seperate punctuations and letters by adding space before and after each punctuation
    translate_table = dict((ord(char), ' ' + char + ' ') for char in string.punctuation)
    # create a map to get tokens
    text = text[0].translate(translate_table)
    # return the tokens after splitting by whitespace
    return text.lower().split()

# remove punctuations
def punc_rem(text):
    # create a string of punctuations that need to be removed
    punc_list = ',!"#$%&()*+/:;<=>@[\\]^`{|}~\t\n'
    # remove punctuations as required
    text = [w for w in text if w not in punc_list]
    return text

# remove stopwords
def stop_word_rem(text):
    # using stopwords offered by NLTK
    text = [ w for w in text if w not in stopwords.words('english')]
    return text

# return a list with stopwords
def clean_with_stopwords(text):
    text = token_with_punc(text)
    text = punc_rem(text)
    return text

# return a list without stopwords
def clean_no_stopwords(text):
    text = clean_with_stopwords(text)
    text = stop_word_rem(text)
    return text

In [21]:
df_pos = pd.read_csv("/Users/mgh/MGH/CE/Waterloo/MSCI_641_NLP/Text-Analytics/pos.txt",header=None, sep="\n")
df_neg = pd.read_csv("/Users/mgh/MGH/CE/Waterloo/MSCI_641_NLP/Text-Analytics/neg.txt",header=None, sep="\n")
frames = [df_pos,df_neg]
df = pd.concat(frames).values

In [43]:
token_with_stopwords = []
for doc in df:
    doc = clean_with_stopwords(doc)
    token_with_stopwords.append(doc)

In [44]:
token_with_stopwords[1:5]

[['she',
  'has',
  'already',
  'tried',
  'one',
  'recipe',
  'a',
  'day',
  'after',
  'receiving',
  'the',
  'book',
  '.'],
 ['i',
  'bought',
  'this',
  'zoku',
  'quick',
  'pop',
  'for',
  'my',
  'daughterr',
  'with',
  'her',
  'zoku',
  'quick',
  'maker',
  '.'],
 ['she',
  'loves',
  'it',
  'and',
  'have',
  'fun',
  'to',
  'make',
  'her',
  'own',
  'ice',
  'cream',
  '.'],
 ['i',
  'was',
  'hoping',
  'there',
  'were',
  'more',
  'where',
  'those',
  'came',
  'from',
  '.']]

In [45]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers =2
                    )

In [46]:
w2v_model.build_vocab(token_with_stopwords, progress_per=10000)

In [47]:
w2v_model.train(token_with_stopwords, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(146556806, 411857280)

In [48]:
w2v_model.init_sims(replace=True)

In [49]:
w2v_model.wv.most_similar(positive=["good"],topn=20)

[('great', 0.7956719398498535),
 ('nice', 0.6755967140197754),
 ('decent', 0.6724581122398376),
 ('excellent', 0.6036227941513062),
 ('fantastic', 0.5993865132331848),
 ('wonderful', 0.5628539323806763),
 ('amazing', 0.5339518189430237),
 ('terrific', 0.5239437818527222),
 ('bad', 0.5177990198135376),
 ('superb', 0.5166008472442627),
 ('fabulous', 0.4999239444732666),
 ('awesome', 0.4963359534740448),
 ('reasonable', 0.49632614850997925),
 ('well', 0.4856538474559784),
 ('impressive', 0.4832674562931061),
 ('perfect', 0.4740520715713501),
 ('outstanding', 0.46723324060440063),
 ('poor', 0.4488271474838257),
 ('exceptional', 0.4462299942970276),
 ('ok', 0.43502235412597656)]

In [50]:
w2v_model.wv.most_similar(positive=["bad"],topn=20)

[('terrible', 0.5330793857574463),
 ('horrible', 0.5330667495727539),
 ('awful', 0.5190057754516602),
 ('good', 0.5177990198135376),
 ('funny', 0.4361669719219208),
 ('poor', 0.4178902804851532),
 ('weird', 0.4173131585121155),
 ('nasty', 0.4119377136230469),
 ('strange', 0.40533795952796936),
 ('gross', 0.39210638403892517),
 ('shabby', 0.3865605592727661),
 ('ok', 0.38646817207336426),
 ('scary', 0.3818008303642273),
 ('okay', 0.37603986263275146),
 ('hideous', 0.37410056591033936),
 ('horrid', 0.3651323914527893),
 ('disappointing', 0.35796576738357544),
 ('stupid', 0.35405805706977844),
 ('worse', 0.3531869053840637),
 ('wierd', 0.3439233899116516)]